These are the functions to run a single N-body simulation. Choose parameters at the top

In [1]:
import pandas as pd
import rebound
import numpy as np
from numpy.random import random
from numpy.random import rayleigh

mjup = 9.543e-4
escale = 0.01 # for Rayleigh distribution
iscale = 1.5*np.pi/180. # 1.5 is about what it's in Fabrycky et al. 2014
tmax=1.e6 # how long do we want to integrate (in units of inner planet orbits)? Shoudl we make it longer for lower mass planets?

def a(planet):
    return ((planet['pl_orbper']/365.25)**2*planet['st_mass'])**(1./3.)
def M(planet):
    return planet['pl_radj']**2*mjup

In [2]:
def run(system):
    starmass = system.head(1)['st_mass'].values[0]
    host = system.head(1)['pl_hostname'].values[0]
    Nplanets = system.head(1)['pl_pnum'].values[0]
    sim = rebound.Simulation()
    sim.G = 4*np.pi**2
    sim.add(m=starmass)
    for i, planet in system.iterrows():
        sim.add(m=M(planet),a=a(planet), e=rayleigh(scale=escale), pomega = random(), inc=rayleigh(scale=iscale), Omega = random(), f = random())

    if sim.N != Nplanets+1: # in some entries, not all planets in the system are in the database, so skip these
        return
    P1 = sim.particles[1].P
    sim.integrator="whfast"
    sim.move_to_com()
    sim.dt = P1*0.07
    sim.initSimulationArchive("binaries/"+host+".bin", interval=P1*1.e5)
    sim.integrate(tmax*P1)    

Load the database

In [3]:
df = pd.read_csv("planets.csv", header=386, index_col=0, skip_blank_lines=True)
df.head()

,pl_hostname,pl_letter,pl_discmethod,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,st_bmyblend,st_m1,st_m1err,st_m1lim,st_m1blend,st_c1,st_c1err,st_c1lim,st_c1blend,st_colorn
rowid,,,,,,,,,,,,,,,,,,,,,
1,11 Com,b,Radial Velocity,1,326.03,0.32,-0.32,0.0,1.290,0.050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,11 UMi,b,Radial Velocity,1,516.22,3.25,-3.25,0.0,1.540,0.070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3,14 And,b,Radial Velocity,1,185.84,0.23,-0.23,0.0,0.830,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
4,14 Her,b,Radial Velocity,1,1773.40,2.50,-2.50,0.0,2.770,0.050,...,0.0,0.366,0.002,0.0,0.0,0.438,0.006,0.0,0.0,9
5,16 Cyg B,b,Radial Velocity,1,798.50,1.00,-1.00,0.0,1.681,0.097,...,0.0,0.222,0.003,0.0,0.0,0.351,0.003,0.0,0.0,17


In [4]:
df.shape

(3431, 379)

Now select system we want to integrate. Put in additional criteria to cut by, separating each by a '&' like below

In [5]:
mask = (df['pl_pnum'] > 1) & (df['pl_radj'] < 1) & (~df['st_mass'].isnull())

In [6]:
df = df[mask]
df.shape

(898, 379)

Choose one of them as an example

In [7]:
host = df['pl_hostname'].unique()[1]
host

'CoRoT-24'

Make a dataframe with just the planets in that system

In [8]:
system = df[df['pl_hostname'] == host]
system.head()

,pl_hostname,pl_letter,pl_discmethod,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,st_bmyblend,st_m1,st_m1err,st_m1lim,st_m1blend,st_c1,st_c1err,st_c1lim,st_c1blend,st_colorn
rowid,,,,,,,,,,,,,,,,,,,,,
76,CoRoT-24,b,Transit,2,5.1134,0.0006,-0.0006,0.0,0.056,0.002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
77,CoRoT-24,c,Transit,2,11.7590,0.0063,-0.0063,0.0,0.098,0.003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


This does the N-body integration, and saves it to a file in the same directory with name hostname.bin

In [9]:
%%time 
run(system)

CPU times: user 12.8 s, sys: 0 ns, total: 12.8 s
Wall time: 12.9 s


Do another system, this time one exhibiting TTVs

In [10]:
maskttv = mask & (df['pl_ttvflag'] == True)
df = pd.read_csv("planets.csv", header=386, index_col=0, skip_blank_lines=True)
df = df[maskttv]
df.shape

(177, 379)

In [11]:
host = df['pl_hostname'].unique()[0]
host

'K2-19'

In [12]:
system = df[df['pl_hostname'] == host]
system.head()

,pl_hostname,pl_letter,pl_discmethod,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,st_bmyblend,st_m1,st_m1err,st_m1lim,st_m1blend,st_c1,st_c1err,st_c1lim,st_c1blend,st_colorn
rowid,,,,,,,,,,,,,,,,,,,,,
723,K2-19,b,Transit,3,7.91940,0.00005,-0.00005,0.0,0.0740,0.0012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
724,K2-19,c,Transit,3,11.90715,0.00150,-0.00150,0.0,0.0971,0.0016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
725,K2-19,d,Transit,3,2.50856,0.00041,-0.00041,0.0,0.0344,0.0006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [13]:
%%time 
run(system)

CPU times: user 22 s, sys: 0 ns, total: 22 s
Wall time: 22.1 s


You could iterate over all the database with the cell below

In [15]:
#for host in df['pl_hostname'].unique():
#    system = df[df['pl_hostname'] == host]
#    run(system)